# Domača naloga 1.1

Rocna izbira metode in portimizacija hiper parametrov 

Za rocno iskanje najboljsega modela sem izbral KNN, SVM in RANDOM FOREST. Podatke sem najprej razdelil v razmerju 1:4 na testne in ucne. Na ucnih sem preverjal tocnost modela z auc metriko (ploscino pod roc krvuljo) z metodo cross validation. Potem pa sem testeral model se na testnih podatkih. Opisan potopek sem najprej izvedel na nepreciscenih podatkih, potem pa se na precisceninih(skaliranih)


In [5]:
import pandas as pd
import numpy as np
import warnings

# ne izpisuje opozoril
warnings.simplefilter(action='ignore', category=FutureWarning)


data = pd.read_csv('podatki.csv')
X = data.drop('y', axis=1)
Y = data['y']
clas = Y.unique()
Y = Y.replace(clas[0], 0)
Y = Y.replace(clas[1], 1)

#print(Y)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)


## KNN

Pri knn sem preveril st. sosedov od 1-30

In [8]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate



dt = {'k':[], 'auc_cv':[], 'auc_test':[], 'auc_cv_sc':[], 'auc_test_sc':[]}
KNN = pd.DataFrame(dt)

k_all = list(range(1, 31))

for k in k_all:
    # not scaled
    knn = KNeighborsClassifier(n_neighbors=k)
    cv = cross_validate(knn, X_train, y_train, cv=5,
    scoring=('roc_auc'),
    return_train_score=True)
    knn.fit(X_train,y_train)
    pred = knn.predict(X_test)
    # scaled
    knn_sc = KNeighborsClassifier(n_neighbors=k)
    cv_sc = cross_validate(knn_sc, X_train_sc, y_train, cv=5,
    scoring=('roc_auc'),
    return_train_score=True)
    knn_sc.fit(X_train_sc, y_train)
    pred_sc = knn_sc.predict(X_test_sc)

    dtk = pd.DataFrame({'k':[k], 'auc_cv':[np.mean(cv['test_score'])], 'auc_test':[roc_auc_score(y_test, pred)],
                        'auc_cv_sc':[np.mean(cv_sc['test_score'])], 'auc_test_sc':[roc_auc_score(y_test, pred_sc)]})
    KNN = KNN.append(dtk)
#print(orgKNN)



In [28]:
KNN = KNN.reset_index().drop('index', axis=1)

print(KNN)

KNN.to_csv('model_scores/bestKNN.csv')

       k    auc_cv  auc_test  auc_cv_sc  auc_test_sc
0    1.0  0.577462  0.601246   0.752869     0.791277
1    2.0  0.585999  0.495154   0.802345     0.652648
2    3.0  0.599292  0.573728   0.833607     0.731222
3    4.0  0.600421  0.474299   0.848437     0.694358
4    5.0  0.604668  0.520509   0.851298     0.705694
5    6.0  0.606421  0.502163   0.856470     0.687349
6    7.0  0.604492  0.516009   0.851171     0.703531
7    8.0  0.609300  0.509173   0.850480     0.654984
8    9.0  0.616101  0.516009   0.858678     0.671166
9   10.0  0.619169  0.490654   0.862626     0.636466
10  11.0  0.628854  0.490654   0.865310     0.671166
11  12.0  0.637134  0.497664   0.866931     0.617947
12  13.0  0.651255  0.495327   0.865489     0.671166
13  14.0  0.647035  0.497664   0.867470     0.654984
14  15.0  0.647745  0.497664   0.867298     0.652648
15  16.0  0.643978  0.500000   0.868503     0.641139
16  17.0  0.645473  0.500000   0.872357     0.652648
17  18.0  0.645376  0.500000   0.876057     0.

In [33]:
bestknn = KNN.idxmax()
#print(bestknn['auc_cv'])
print(f"Best AUC of cv {KNN['auc_cv'][bestknn['auc_cv']]} with k={bestknn['auc_cv']+1} and corresponding test AUC {KNN['auc_test'][bestknn['auc_cv']]}")
print(f"Best AUC of cv scaled {KNN['auc_cv_sc'][bestknn['auc_cv_sc']]} with k={bestknn['auc_cv_sc']+1} and corresponding test AUC {KNN['auc_test_sc'][bestknn['auc_cv_sc']]}")



Best AUC of cv 0.6513123605827611 with k=22 and corresponding test 0.5
Best AUC of cv scaled 0.8788046786369016 with k=22 and corresponding test 0.6388023537556248


## SVM

Pri podpornih vektorjih sem preveril linearno, polinomsko in sigmoidno jedo. Za paramater komplesnosti `C` sem stestiral 1 in 10

In [ ]:
from sklearn.svm import SVC


dtsvm = {'kernel':[], 'C':[], 'auc_cv':[], 'auc_test':[], 'auc_cv_sc':[], 'auc_test_sc':[]}
SVM = pd.DataFrame(dtsvm)

c_all = [1,10]
kernels = ['linear', 'poly', 'sigmoid']

for c in c_all:
    for kernel in kernels:
        # not scaled
        svm = SVC(kernel=kernel, C=c, random_state=42)
        cv = cross_validate(svm, X_train, y_train, cv=5,
            scoring=('roc_auc'),
            return_train_score=True)
        svm.fit(X_train,y_train)
        pred = svm.predict(X_test)

        # scaled
        svm_sc = SVC(kernel=kernel, C=c, random_state=42)
        cv_sc = cross_validate(svm_sc, X_train_sc, y_train, cv=5,
            scoring=('roc_auc'),
            return_train_score=True)
        svm_sc.fit(X_train_sc, y_train)
        pred_sc = svm_sc.predict(X_test_sc)

        dti = pd.DataFrame({'kernel':[kernel],'C':[c] , 'auc_cv':[np.mean(cv['test_score'])], 'auc_test':[roc_auc_score(y_test, pred)],
                            'auc_cv_sc':[np.mean(cv_sc['test_score'])], 'auc_test_sc':[roc_auc_score(y_test, pred_sc)]})
        SVM = SVM.append(dti)


In [32]:
SVM = SVM.reset_index().drop('index', axis=1)
print(SVM)
SVM.to_csv('model_scores/bestSVM.csv')

    kernel     C    auc_cv  auc_test  auc_cv_sc  auc_test_sc
0   linear   1.0  0.785345  0.745067   0.882919     0.705867
1     poly   1.0  0.598600  0.518519   0.872821     0.620284
2  sigmoid   1.0  0.369386  0.541364   0.661341     0.652302
3   linear  10.0  0.773971  0.691848   0.885612     0.708204
4     poly  10.0  0.471172  0.537037   0.840054     0.652648
5  sigmoid  10.0  0.368012  0.541364   0.616342     0.693839


In [41]:
bestsvm = SVM.drop('kernel', axis=1).idxmax()
#print(bestsvm)
print(f"""
Best AUC of cv {SVM['auc_cv'][bestsvm['auc_cv']]} 
with C={SVM['C'][bestsvm['auc_cv']]} kernel={SVM['kernel'][bestsvm['auc_cv']]}
and corresponding test AUC {SVM['auc_test'][bestsvm['auc_cv']]}""")
print(f"""
Best AUC of cv scaled {SVM['auc_cv_sc'][bestsvm['auc_cv_sc']]} 
with C={SVM['C'][bestsvm['auc_cv_sc']]} kernel={SVM['kernel'][bestsvm['auc_cv_sc']]} 
and corresponding test AUC {SVM['auc_test_sc'][bestsvm['auc_cv_sc']]}""")



Best AUC of cv 0.7853449199799003 
with C=1.0 kernel=linear
and corresponding test AUC 0.7450674974039461

Best AUC of cv scaled 0.885611754089776 
with C=10.0 kernel=linear 
and corresponding test AUC 0.7082035306334372


## Random forest

Optimizacija hiperparametrov: izbira max_depth = [2, 7, 12, 17, 22, 27, 32, 37, 42, 47] in min_samples_split = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

In [12]:
from sklearn.ensemble import RandomForestClassifier


dt = {'max_depth':[],'min_samples_split':[], 'auc_cv':[], 'auc_test':[], 'auc_cv_sc':[], 'auc_test_sc':[]}
RF = pd.DataFrame(dt)

max_depths = list(range(2, 50, 5))
min_sampls = list(range(1, 20, 2))


for max_d in max_depths:
    for min_s in min_sampls:
        # not scaled
        rf = RandomForestClassifier(max_depth=max_d,min_samples_split=min_s)
        cv = cross_validate(rf, X_train, y_train, cv=5,
        scoring=('roc_auc'),
        return_train_score=True)
        rf.fit(X_train,y_train)
        pred = rf.predict(X_test)
        # scaled
        rf_sc = RandomForestClassifier(max_depth=max_d,min_samples_split=min_s)
        cv_sc = cross_validate(rf_sc, X_train_sc, y_train, cv=5,
        scoring=('roc_auc'),
        return_train_score=True)
        rf_sc.fit(X_train_sc, y_train)
        pred_sc = rf_sc.predict(X_test_sc)

        dti = pd.DataFrame({'max_depth':[max_d],'min_samples_split':[min_s], 
                            'auc_cv':[np.mean(cv['test_score'])], 'auc_test':[roc_auc_score(y_test, pred)],
                            'auc_cv_sc':[np.mean(cv_sc['test_score'])], 'auc_test_sc':[roc_auc_score(y_test, pred_sc)]})
        RF = RF.append(dti)

In [39]:
RF = RF.reset_index().drop('index', axis=1)
print(RF)
RF.to_csv('model_scores/bestRF.csv')

    max_depth  min_samples_split    auc_cv  auc_test  auc_cv_sc  auc_test_sc
0         2.0                1.0  0.882948  0.500000   0.882112     0.500000
1         2.0                3.0  0.877291  0.500000   0.879778     0.500000
2         2.0                5.0  0.882490  0.500000   0.883998     0.500000
3         2.0                7.0  0.878572  0.500000   0.883029     0.500000
4         2.0                9.0  0.878305  0.500000   0.882696     0.500000
..        ...                ...       ...       ...        ...          ...
95       47.0               11.0  0.926973  0.643475   0.926054     0.694358
96       47.0               13.0  0.929814  0.659657   0.926484     0.673503
97       47.0               15.0  0.927204  0.661994   0.923801     0.694358
98       47.0               17.0  0.922884  0.694358   0.922863     0.675839
99       47.0               19.0  0.928968  0.678176   0.920293     0.675839

[100 rows x 6 columns]


In [43]:
bestrf = RF.idxmax()
#print(bestrf)
print(f"""
Best AUC of cv {RF['auc_cv'][bestrf['auc_cv']]} 
with max_depth={RF['max_depth'][bestrf['auc_cv']]} min_samples_split={RF['min_samples_split'][bestrf['auc_cv']]}
and corresponding test AUC {RF['auc_test'][bestrf['auc_cv']]}""")
print(f"""
Best AUC of cv scaled {RF['auc_cv_sc'][bestrf['auc_cv_sc']]} 
with max_depth={RF['max_depth'][bestrf['auc_cv_sc']]} min_samples_split={RF['min_samples_split'][bestrf['auc_cv_sc']]} 
and corresponding test AUC {RF['auc_test_sc'][bestrf['auc_cv_sc']]}""")


Best AUC of cv 0.933307194764305 
with max_depth=32.0 min_samples_split=5.0
and corresponding test AUC 0.6966943579093112

Best AUC of cv scaled 0.9322728420308515 
with max_depth=27.0 min_samples_split=1.0 
and corresponding test AUC 0.7337313949463482


### Izbira najbolsega modela

Njaboljsi model sem izbral na podlagi najboljsega rezultata metrike AUC (ploscina pod ROC krivuljo). Izkazalo se je da je skaliranje podatkov pripomoglo k boljsim rezultatom, razen pri RF so bili rezultati dokaj podobni.

In [56]:
models = pd.DataFrame(data={
    'model':['KNN', 'SVM', 'RF'],
    'bestAUC':[KNN['auc_cv_sc'][bestknn['auc_cv_sc']], 
               SVM['auc_cv_sc'][bestsvm['auc_cv_sc']], 
               RF['auc_cv_sc'][bestrf['auc_cv_sc']]],
    'params':[f"{KNN['k'][bestknn['auc_cv_sc']]}", 
              f"C={SVM['C'][bestsvm['auc_cv_sc']]} & kernel={SVM['kernel'][bestsvm['auc_cv_sc']]}", 
              f"max_depth={RF['max_depth'][bestrf['auc_cv_sc']]} & min_samples_split={RF['min_samples_split'][bestrf['auc_cv_sc']]}"]
        }
    ).set_index('model')

bestmodel = models.drop('params', axis=1).idxmax()
print(f"""
Best model is {bestmodel['bestAUC']} 
with AUC score {models['bestAUC'][bestmodel['bestAUC']]} 
and params {models['params'][bestmodel['bestAUC']]}""")


Best model is RF 
with AUC score 0.9322728420308515 
and params max_depth=27.0 & min_samples_split=1.0
